In [ ]:
import os
os.chdir("../")
os.getcwd()

In [ ]:
from attacker.ResNet34 import *
from attacker.query import *
from attacker.utils import *

set_seed(42)

# REF: https://towardsdev.com/implement-resnet-with-pytorch-a9fb40a77448

# Query victim model & Train attacker model

Uses default configuration info

In [ ]:
# get dataset in dataloader
trainloader, testloader = getDataset(config["victim"]["data"], config['batch_size'])

# query & save test data
querytestloader = QueryVictim(config["victim"], testloader, len(testloader.dataset))
#torch.save(querytestloader, f'attacker/queried_data/{config["victim"]["data"]}_queriedtestdataloader.pt')

# query train data
querytrainloader = QueryVictim(config["victim"], trainloader, config["query_size"], query_type=config["query_type"])

# initialize attacker model
attacker = ResNet34(3, ResBlock, outputs=10)
attacker_loss = torch.nn.CrossEntropyLoss()

# train attacker model
attacker_result = Training(attacker, querytrainloader, querytestloader, (-1, 3, 32, 32), config["epochs"], optimizer=torch.optim.Adam(
    attacker.parameters(), lr=config["learning_rate"]), loss=attacker_loss)

# save & visualize model inference
percent = SaveVisualize(attacker, attacker_result,  f'Trial-A_R34_{config["victim"]["model_name"]}_{config["victim"]["data"]}{config["query_type"]}_{config["query_size"]}')
results_dict={"Victim":config["victim"]["model_name"],
                "Dataset":config["victim"]["data"],
                "QueryType": config["query_type"],
                "QuerySize": config["query_size"],
                "Attacker": 'resnet34',
                "Train Loss": percent[0],
                "Train Accuracy": percent[1],
                "Test Loss": percent[2],
                "Test Accuracy": percent[3]}
print(results_dict)

# Investigations

In [ ]:
# Iterate through Model & Dataset
Results = []
for victim in parameters["victim"]:
    print('---------------------------------------------------------------------------')
    # get dataset in dataloader
    trainloader, testloader = getDataset(victim["data"], config['batch_size'])
    
    # query test data
    querytestloader = QueryVictim(victim, testloader, len(testloader.dataset))
    
    # Iterate Through Query Type
    for querytype in parameters["query_type"]:
        # Iterate Through Query Size
        for size in parameters["query_size"]:
            print('-----------------------------------------------------------------------------')
            print(f'--------Victim: {victim["model_name"]} Dataset: {victim["data"]}------------')
            print(f'---------------Query Type: {querytype} Query Size: {size}-------------------')
            
            # query train data
            querytrainloader = QueryVictim(victim, trainloader, size, query_type=querytype)
            
            # initialize attacker model
            attacker = ResNet34(3, ResBlock, outputs=10)
            attacker_loss = torch.nn.CrossEntropyLoss()
            
            # train attacker model
            attacker_result = Training(attacker, querytrainloader, querytestloader, (-1, 3, 32, 32), config["epochs"], optimizer=torch.optim.Adam(attacker.parameters(), lr=config["learning_rate"]), loss=attacker_loss)
            
            # save & visualize model inference
            percent = SaveVisualize(attacker, attacker_result, f'A_R34_{victim["model_name"]}_{victim["data"]}_{querytype}_{size}')
            results_dict={"Victim":victim["model_name"],
                            "Dataset":victim["data"],
                            "QueryType": querytype,
                            "QuerySize": size,
                            "Attacker": 'resnet34',
                            "Train Loss": percent[0],
                            "Train Accuracy": percent[1],
                            "Test Loss": percent[2],
                            "Test Accuracy": percent[3]}

            print(results_dict)
            Results.append(results_dict)
            print('-----------------------------------------------------------------------------')

In [ ]:
Results


# Archived

Load querytestloader from file

In [ ]:
# load queried test data
querytestloader = torch.load(f'attacker/queried_data/{config["victim"]["data"]}_queriedtestdataloader.pt')
print(f'Input dataset:{len(testloader.dataset)}, Queried dataset:{len(querytestloader.dataset)}')

Testing Time outputs

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    qtl = QueryVictim(trainloader, 1000, sampling='coreset_cross')
print(f'Query Time:{prof.key_averages().self_cpu_time_total/1e6}s)

Visualize random Images from CIFAR-10

In [ ]:
class_names = ['air_plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# get some random training images
plt.figure(figsize=(10, 10))
counter = 0
for images, labels in trainloader:
    for i, img in enumerate(images):
        plt.subplot(5, 5, counter+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        img = img/2 + 0.5   # unmornalize
        # convert (C,H,W) to (H,W,C)
        img_show = np.transpose(img, (1, 2, 0))
        plt.imshow(img_show, cmap=plt.cm.binary)
        plt.xlabel(class_names[int(labels[i])], color='red', fontsize='large')
        counter += 1
        if counter == 25:
            break
    if counter == 25:
        break

plt.show()

Train a pseudo victim Model

In [ ]:
# create victim as resnet34
victim = ResNet34(3, ResBlock, outputs=10)
victim_loss = torch.nn.CrossEntropyLoss()

# train model
victim_result = Training(victim, trainloader, testloader, (-1, 3, 32, 32), config["epochs"], optimizer=torch.optim.Adam(
    victim.parameters(), lr=config["learning_rate"]), loss=victim_loss)

# save model inference
torch.save(victim.state_dict(), 'Victim_resnet34')

# visualization
plots(victim_result, 'Victim-resnet34')

Load Victim from File

In [ ]:
victim = ResNet34(3, ResBlock, outputs=10)
victim.load_state_dict(torch.load('Victim_resnet34'))
victim.eval()